# Importando libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, Dropdown
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path
import re
import numpy as np
import json
from ipyfilechooser import FileChooser
from IPython.display import display
from tkinter import Tk, filedialog
from pathlib import Path

# Lendo o dataset

In [2]:
try:   
   df_processed = pd.read_csv('../data/prouni_2005_2019_processed.csv')
except Exception as e:
    print(f'Error: {e}')

#df_processed.head()

In [3]:
#df_processed

In [4]:
df_chars = df_processed.copy()

In [5]:
lista_colunas = df_processed.columns
lista_colunas

Index(['ANO_CONCESSAO', 'INSTITUICAO', 'TIPO', 'MODALIDADE_ENSINO', 'CURSO',
       'TURNO', 'SEXO', 'RACA', 'DEFICIENTE_FISICO', 'REGIAO', 'UF',
       'MUNICIPIO', 'IDADE'],
      dtype='object')

In [6]:
#lista_renomeada = [col.replace('_', ' ').upper() for col in lista_colunas]
lista_renomeada = ['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA','MODALIDADE DE ENSINO','CURSO','TURNO','SEXO','RAÇA','DEFICIENTE FÍSICO','REGIÃO','UF','MUNICÍPIO','IDADE']
df_chars.rename(columns=dict(zip(lista_colunas, lista_renomeada)), inplace=True)
df_chars.columns

Index(['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA',
       'MODALIDADE DE ENSINO', 'CURSO', 'TURNO', 'SEXO', 'RAÇA',
       'DEFICIENTE FÍSICO', 'REGIÃO', 'UF', 'MUNICÍPIO', 'IDADE'],
      dtype='object')

In [7]:
df_chars = df_chars[df_chars['TIPO DE BOLSA'] != 'BOLSA COMPLEMENTAR 25%']
#df_chars

In [8]:
sum_bolsa_25 = (df_chars['TIPO DE BOLSA'] == 'BOLSA COMPLEMENTAR 25%').sum()
print(f'Total bolsa 25%: {sum_bolsa_25}')

Total bolsa 25%: 0


# Gráficos

### Funções para criar botão de salvar gráfico em png

In [9]:
def limpar_nome_arquivo(name: str, max_len: int = 160) -> str:
    name = (name or 'grafico').strip()
    name = re.sub(r'\s+', ' ', name)
    name = re.sub(r'[\\/:*?\'<>|]+', '_', name)  # proibidos no Windows
    name = name.strip(' ._')
    if not name:
        name = 'grafico'
    return name[:max_len]

def salvar_plotly_png(fig, folder: str, default_title: str = 'grafico') -> str:
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)

    title_text = getattr(getattr(getattr(fig, 'layout', None), 'title', None), 'text', None) or default_title
    base = limpar_nome_arquivo(str(title_text))

    # troca espaços por underscore só no nome do arquivo
    base_fs = base.replace(' ', '_')

    outpath = folder_path / f'{base_fs}.png'
    i = 2
    while outpath.exists():
        outpath = folder_path / f'{base_fs}_{i}.png'
        i += 1

    fig.write_image(str(outpath))
    return str(outpath)

def fazer_plotly_widget_com_salvar(
    plot_fn,
    controls,
    folder,
    default_title='grafico',
    run_once=True,
):
    out_plot = widgets.Output()
    btn = widgets.Button(description='Salvar PNG', icon='save')
    msg = widgets.HTML('')
    state = {'fig': None}

    def _renderizar(change=None):
        with out_plot:
            out_plot.clear_output(wait=True)
            kwargs = {k: w.value for k, w in controls.items()}
            fig = plot_fn(**kwargs)
            state['fig'] = fig
            if fig is None:
                print('Nenhum gráfico para mostrar.')
                return
            fig.show()

    def _on_salvar(_):
        fig = state['fig']
        if fig is None:
            msg.value = "<span style='color:#b00'>Ainda não existe gráfico para salvar.</span>"
            return
        try:
            path = salvar_plotly_png(fig, folder, default_title=default_title)
            msg.value = f'<span style="color:#060">Salvo em: {path}</span>'
        except Exception as e:
            msg.value = f'<span style="color:#b00">Erro ao salvar: {e}</span>'

    btn.on_click(_on_salvar)

    for w in controls.values():
        w.observe(_renderizar, names='value')

    ui = widgets.HBox([*controls.values(), btn, msg])

    if run_once:
        _renderizar()

    display(ui, out_plot)

    #return {'ui': ui, 'out': out_plot, 'button': btn, 'message': msg, 'state': state}


### Copiando e renomeando colunas

In [10]:
df_chars['IDADE'].describe()

count    2.683451e+06
mean     3.135969e+01
std      7.949649e+00
min      1.500000e+01
25%      2.500000e+01
50%      3.000000e+01
75%      3.500000e+01
max      9.000000e+01
Name: IDADE, dtype: float64

In [11]:
df_chars['FAIXA DE IDADE'] = pd.cut(
    df_chars['IDADE'],
    bins=[0, 18, 30, 40, 59, float('inf')],
    labels=['ADOLESCENTE (ATÉ 18)', 'ADULTO JOVEM (19-30)', 'ADULTO (31-40)','MEIA IDADE (41-59)', 'IDOSO (60+)'],
    right=False
)

#df_chars['FAIXA DE IDADE'].value_counts()

ordem = ['ADOLESCENTE (ATÉ 18)', 'ADULTO JOVEM (19-30)', 'ADULTO (31-40)','MEIA IDADE (41-59)', 'IDOSO (60+)']

faixas = [f for f in ordem if f in df_chars['FAIXA DE IDADE'].unique()]


### Criando dropdowns

In [12]:
tipos = ['TODOS OS TIPOS'] + sorted(df_chars['TIPO DE BOLSA'].unique())
anos = ['TODOS OS ANOS']
anos.extend(sorted(df_chars['ANO DE CONCESSÃO'].unique()))
regioes = ['PAÍS INTEIRO', 'COMPARAR REGIÕES'] + sorted(df_chars['REGIÃO'].unique())
modos = ['QUANTIDADE', 'VARIAÇÃO DO PERÍODO (%)', 'CRESCIMENTO PERCENTUAL']

In [13]:
lista_anos_numeros = list(np.array(anos)[1:])
print(lista_anos_numeros)

[np.str_('2005'), np.str_('2006'), np.str_('2007'), np.str_('2008'), np.str_('2009'), np.str_('2010'), np.str_('2011'), np.str_('2012'), np.str_('2013'), np.str_('2014'), np.str_('2015'), np.str_('2016'), np.str_('2017'), np.str_('2018'), np.str_('2019')]


In [14]:
dropdown_tipo = widgets.Dropdown(
    options=tipos, value=tipos[0],
    description='TIPO DE BOLSA'
)


dropdown_modo = widgets.Dropdown(
    options=modos,
    value='QUANTIDADE',
    description='ESCALA'
)
dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description='ANO'
)

dropdown_faixa = widgets.Dropdown(
    options=['TODAS AS IDADES'] + faixas,
    value='TODAS AS IDADES',
    description='FAIXA DE IDADE'
)

dropdown_regiao = widgets.Dropdown(
    options=regioes,
    value='PAÍS INTEIRO',
    description='REGIÃO'
)

periodos = [
    ('ANUAL (1 ANO)', 1),
    ('TRIÊNIO (3 ANOS)', 3),
    ('QUINQUÊNIO (5 ANOS)', 5),
]

dropdown_periodo = widgets.Dropdown(
    options=periodos,
    value=1,
    description='PERÍODO'
)
dropdown_periodo.style = {'description_width': 'initial'}
dropdown_periodo.layout = widgets.Layout(width='220px')

dropdown_ano.style = {'description_width': 'initial'}
dropdown_faixa.style = {'description_width': 'initial'}
dropdown_modo.style = {'description_width': 'initial'}
dropdown_regiao.style = {'description_width': 'initial'}
dropdown_tipo.style = {'description_width': 'initial'}

dropdown_ano.layout = widgets.Layout(width='180px')
dropdown_faixa.layout = widgets.Layout(width='290px')
dropdown_tipo.layout = widgets.Layout(width='320px')
dropdown_regiao.layout = widgets.Layout(width='230px')
dropdown_modo.layout = widgets.Layout(width='280px')

## Número de bolsas por idade, tipo e gênero

In [15]:
def total_idade_tipo_e_sexo(ano, faixa_idade):
    base = df_chars[df_chars['ANO DE CONCESSÃO'] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_chars

    if faixa_idade != 'TODAS AS IDADES':
        base = base[base['FAIXA DE IDADE'].astype(str) == faixa_idade]

    agrupado = (
        base.groupby(['TIPO DE BOLSA', 'SEXO'])
            .size()
            .reset_index(name='QUANTIDADE')
    )

    tabela = agrupado.pivot_table(
        index='TIPO DE BOLSA',
        columns='SEXO',
        values='QUANTIDADE',
        fill_value=0,
    ).reset_index()

    if 'F' not in tabela.columns:
        tabela['F'] = 0
    if 'M' not in tabela.columns:
        tabela['M'] = 0
    
    tabela['TOTAL'] = tabela['F'] + tabela['M']

    tabela = tabela.rename(columns={
        'F': 'FEMININO',
        'M': 'MASCULINO'
    })
    
    if tabela.empty:
        print('Sem dados para esse filtro.')
        return
    
    titulo = f'Quantidade de bolsas por sexo e tipo de bolsa de 2005 a 2019' if ano == 'TODOS OS ANOS' else f'Quantidade de bolsas por sexo e tipo de bolsa em {ano}'   
    
    if faixa_idade != 'TODAS AS IDADES':
        titulo += f' entre {faixa_idade.lower().replace('adulto', 'adultos').replace('jovem', 'jovens').replace('idoso', 'idosos')}'
    
    fig = px.bar(
        tabela,
        x='TIPO DE BOLSA',
        y=['TOTAL', 'FEMININO', 'MASCULINO'],
        barmode='group',
        title=titulo,
        text_auto=True,
        color_discrete_map={
            'TOTAL': 'green',
            'FEMININO': 'magenta',
            'MASCULINO': 'blue'
        }
    )

    fig.update_traces(
    hovertemplate=
        'TIPO: %{x}<br>'
        + 'QTDE: %{y:,.0f}'.replace(',', '.')
        + '<extra></extra>',
        textposition='outside',
        texttemplate='%{y:,.0f}'.replace(',', '.')
    )
    
    fig.update_layout(
        height=600,
        width=1000,
        xaxis_title='TIPO DE BOLSA',
        yaxis_title='QUANTIDADE DE BOLSAS',
        uniformtext_minsize=6,
        uniformtext_mode='hide',
        legend_title='LEGENDA',
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black', size=14),
        separators=',.'
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=total_idade_tipo_e_sexo,
    controls={'ano': dropdown_ano, 'faixa_idade': dropdown_faixa},
    folder=f'../figures/{total_idade_tipo_e_sexo.__name__}',
    default_title='total_idade_tipo_e_sexo'
)

Output()

In [16]:
def total_idade_tipo_e_sexo_animado():

    base = df_chars.copy()

    base['SEXO'] = base['SEXO'].map({'F': 'FEMININO', 'M': 'MASCULINO'})
    base_total = base.copy()
    base_total['SEXO'] = 'TOTAL'

    base = pd.concat([base, base_total], ignore_index=True)

    # Agrupar por ano, faixa de idade e sexo
    agrupado = (
        base
        .groupby(['ANO DE CONCESSÃO', 'FAIXA DE IDADE', 'SEXO'])
        .size()
        .reset_index(name='QUANTIDADE DE BOLSAS')
    )

    #agrupado.head()


    # Gráfico em Plotly com slider de ano e barras agrupadas por sexo
    fig = px.bar(
        agrupado,
        x='FAIXA DE IDADE',
        y='QUANTIDADE DE BOLSAS',
        color='SEXO',
        animation_frame='ANO DE CONCESSÃO',
        barmode='group',
        labels={
            'FAIXA DE IDADE': 'Faixa de idade',
            'QUANTIDADE DE BOLSAS': 'Quantidade de bolsas',
            'SEXO': 'Sexo',
            'ANO DE CONCESSÃO': 'Ano de concessão'
        },
        color_discrete_map={
            'TOTAL': 'green',
            'FEMININO': 'magenta',
            'MASCULINO': 'blue'
        },
        title='Bolsas do Prouni por faixa de idade, ano de concessão e sexo'
    )
    
    max_y = agrupado['QUANTIDADE DE BOLSAS'].max() 

    fig.update_yaxes(range=[0, max_y * 1.1])

    
    fig.update_layout(
        xaxis_tickangle=-45,
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        height=1000,
        uniformtext_minsize=6,
        uniformtext_mode='show',
        legend_title='Sexo',
    )
    fig.show()

total_idade_tipo_e_sexo_animado()

C:\Users\felip\AppData\Local\Temp\ipykernel_17112\1567575110.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [17]:
def grafico_crescimento_por_periodo_sexo(
    faixa_idade,
    tipo_bolsa,
    modo,
    periodo=3,                 # 1=ano, 3=triênio, 5=quinquênio...
    ano_col='ANO DE CONCESSÃO',
    sexo_col='SEXO',
    faixa_col='FAIXA DE IDADE',
    tipo_col='TIPO DE BOLSA',
):
    base = df_chars.copy()

    # mapear sexo
    base[sexo_col] = base[sexo_col].map({'F': 'FEMININO', 'M': 'MASCULINO'})

    # filtros
    if faixa_idade != 'TODAS AS IDADES':
        base = base[base[faixa_col].astype(str) == faixa_idade]

    if tipo_bolsa != 'TODOS OS TIPOS':
        base = base[base[tipo_col] == tipo_bolsa]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

    # garante ano como int
    base[ano_col] = base[ano_col].astype(int)

    # define período: início do bloco (ex: 2005, 2008, 2011...)
    ano_min = int(base[ano_col].min())
    base['PERIODO_INICIO'] = ((base[ano_col] - ano_min) // periodo) * periodo + ano_min
    base['PERIODO_LABEL'] = (
        base['PERIODO_INICIO'].astype(str)
        + '-'
        + (base['PERIODO_INICIO'] + (periodo - 1)).astype(str)
    )

    # agrega por sexo e período
    agrupado = (
        base.groupby([sexo_col, 'PERIODO_INICIO', 'PERIODO_LABEL'])
            .size()
            .reset_index(name='QUANTIDADE')
            .sort_values([sexo_col, 'PERIODO_INICIO'])
    )

    # variação percentual entre períodos (vs período anterior)
    agrupado['VARIAÇÃO DO PERÍODO (%)'] = (
        agrupado.groupby(sexo_col)['QUANTIDADE'].pct_change().mul(100)
    )

    # crescimento vs primeiro período
    def crescimento_base(s):
        primeiro = s.iloc[0]
        if primeiro == 0:
            return pd.Series([pd.NA] * len(s), index=s.index)
        return (s / primeiro - 1) * 100

    agrupado['CRESCIMENTO BASE'] = (
        agrupado.groupby(sexo_col)['QUANTIDADE'].transform(crescimento_base)
    )

    # crescimento cumulativo (produto dos fatores)
    def crescimento_cumulativo(s):
        r = s.pct_change()
        fator = (1 + r).fillna(1.0).cumprod()
        return (fator - 1) * 100

    agrupado['CRESCIMENTO PERCENTUAL'] = (
        agrupado.groupby(sexo_col)['QUANTIDADE'].transform(crescimento_cumulativo)
    )

    # escolha da métrica
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    elif modo == 'VARIAÇÃO ANUAL (%)' or modo == 'VARIAÇÃO DO PERÍODO (%)':
        metrica = 'VARIAÇÃO DO PERÍODO (%)'
        y_label = f'Variação vs período anterior (%)'
    else:
        metrica = 'CRESCIMENTO PERCENTUAL'
        y_label = 'Crescimento em relação ao primeiro período (%)'

    # título
    tipo_titulo = (
        'todas as bolsas'
        if tipo_bolsa == 'TODOS OS TIPOS'
        else tipo_bolsa.lower()
            .replace('bolsa', 'bolsas')
            .replace('parcial', 'parciais')
            .replace('complementar', 'complementares')
            .replace('integral', 'integrais')
    )

    periodo_nome = 'ano' if periodo == 1 else f'períodos de {periodo} anos'
    if faixa_idade == 'TODAS AS IDADES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} por {periodo_nome} comparando sexo'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} por {periodo_nome} na faixa etária {faixa_idade} anos'

    fig = px.line(
        agrupado,
        x='PERIODO_LABEL',
        y=metrica,
        color=sexo_col,
        markers=True,
        hover_data={
            'PERIODO_LABEL': True,
            sexo_col: True,
            'QUANTIDADE': ':.0f',
            'VARIAÇÃO DO PERÍODO (%)': ':.1f' if metrica == 'VARIAÇÃO DO PERÍODO (%)' else False,
            'CRESCIMENTO PERCENTUAL': ':.1f' if metrica == 'CRESCIMENTO PERCENTUAL' else False,
        },
        color_discrete_map={
            'FEMININO': 'magenta',
            'MASCULINO': 'blue',
        },
        title=titulo,
    )

    # ordem legenda
    ordem = ['FEMININO', 'MASCULINO']
    fig.data = tuple(sorted(fig.data, key=lambda tr: ordem.index(tr.name)))

    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title='Período',
        yaxis_title=y_label,
        legend_title='Sexo',
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
        width=1200,
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=grafico_crescimento_por_periodo_sexo,
    controls={
        'faixa_idade': dropdown_faixa,
        'tipo_bolsa': dropdown_tipo,
        'modo': dropdown_modo,
        'periodo': dropdown_periodo,
    },
    folder=f'../figures/{grafico_crescimento_por_periodo_sexo.__name__}',
    default_title='crescimento_por_periodo_sexo',
)

Output()

## Número de bolsas por idade, tipo e raça

In [18]:
pd.DataFrame(df_chars['RAÇA']).value_counts()

RAÇA         
Branca           1158633
Parda            1110892
Preta             340823
Amarela            47002
Nao Informada      23237
Indigena            2864
Name: count, dtype: int64

In [19]:
def total_idade_tipo_e_raça(ano, faixa_idade):
    base = df_chars[df_chars['ANO DE CONCESSÃO'] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_chars
    
    if faixa_idade != 'TODAS AS IDADES':
        base = base[base['FAIXA DE IDADE'].astype(str) == faixa_idade]

    agrupado = (
        base.groupby(['TIPO DE BOLSA', 'RAÇA'])
            .size()
            .reset_index(name='QUANTIDADE')
    )

    tabela = agrupado.pivot_table(
        index='TIPO DE BOLSA',
        columns='RAÇA',
        values='QUANTIDADE',
        fill_value=0,
    ).reset_index()

    if 'Branca' not in tabela.columns:
        tabela['Branca'] = 0
    if 'Preta' not in tabela.columns:
        tabela['Preta'] = 0
    if 'Parda' not in tabela.columns:
        tabela['Parda'] = 0
    if 'Amarela' not in tabela.columns:
        tabela['Amarela'] = 0
    if 'Indigena' not in tabela.columns:
        tabela['Indigena'] = 0
    if 'Nao Informada' not in tabela.columns:
        tabela['Nao Informada'] = 0

    tabela['TOTAL'] = tabela['Branca'] + tabela['Preta'] + tabela['Parda'] + tabela['Amarela'] + tabela['Indigena'] + tabela['Nao Informada']

    tabela = tabela.rename(columns={
        'Branca' : 'BRANCA',
        'Preta' : 'PRETA',
        'Parda' : 'PARDA',
        'Amarela' : 'AMARELA',
        'Indigena' : 'INDÍGENA',
        'Nao Informada' : 'NÃO INFORMADA'
    })
    
    if tabela.empty:
        print('Sem dados para esse filtro.')
        return

    titulo = f'Quantidade de bolsas por tipo e raça de 2005 a 2019' if ano == 'TODOS OS ANOS' else f'Quantidade de bolsas por tipo e raça em {ano}'
    
    fig = px.bar(
        tabela,
        x='TIPO DE BOLSA',
        y=['TOTAL', 'BRANCA', 'PRETA', 'PARDA', 'AMARELA', 'INDÍGENA', 'NÃO INFORMADA'],
        barmode='group',
        title=titulo,
        text_auto=True,
        color_discrete_map={
            'TOTAL': 'green',
            'BRANCA': 'cyan',
            'PRETA': 'black',
            'PARDA': 'blue',
            'AMARELA': 'yellow',
            'INDÍGENA': 'red',
            'NÃO INFORMADA': 'magenta'
        }
    )

    # separadores pt-BR: decimal ',' e milhar '.'
    fig.update_layout(separators=',.')

    # cada trace vira uma 'raça', então a gente replica o nome dela para cada barra
    for tr in fig.data:
        nome = 'TOTAL' if tr.name == 'TOTAL' else tr.name
        tr.customdata = [nome] * len(tr.x)

    fig.update_traces(
        hovertemplate='TIPO: %{x}<br>QTDE: %{y:,.0f}<br>RAÇA: %{customdata}<extra></extra>',
        textposition='outside',
        texttemplate='%{y:,.0f}',
        textfont_size=22,
        textfont_color='black',
        textfont_family='Arial Black'
    )

    fig.update_yaxes(
    dtick=200_000
    )

    
    fig.update_layout(
        height=1000,
        width=1400,
        xaxis_title='TIPO DE BOLSA',
        yaxis_title='QUANTIDADE DE BOLSAS',
        uniformtext_minsize=11,
        uniformtext_mode='show',
        legend_title='RAÇA',
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black',)
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=total_idade_tipo_e_raça,
    controls={'ano': dropdown_ano, 'faixa_idade': dropdown_faixa},
    folder=f'../figures/{total_idade_tipo_e_raça.__name__}',
    default_title='total_idade_tipo_e_raça'
)

Output()

In [20]:
def crescimento_idade_tipo_e_raca(
    faixa_idade,
    tipo_bolsa,
    modo,
    periodo=3,                 # 1=ano, 3=triênio, 5=quinquênio...
    ano_col='ANO DE CONCESSÃO',
    faixa_col='FAIXA DE IDADE',
    tipo_col='TIPO DE BOLSA',
    raca_col='RAÇA' ):
    base = df_chars.copy()

    # padroniza raça
    base['RAÇA'] = base['RAÇA'].map({
        'Branca': 'BRANCA',
        'Preta': 'PRETA',
        'Parda': 'PARDA',
        'Amarela': 'AMARELA',
        'Indigena': 'INDÍGENA',
        'Nao Informada': 'NÃO INFORMADA'
    })

    # filtro faixa
    if faixa_idade != 'TODAS AS IDADES':
        base = base[base[faixa_col].astype(str) == faixa_idade]

    if tipo_bolsa != 'TODOS OS TIPOS':
        base = base[base[tipo_col] == tipo_bolsa]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

     # agrega por sexo e ano
    base[ano_col] = base[ano_col].astype(int)

    # define período: início do bloco (ex: 2005, 2008, 2011...)
    ano_min = int(base[ano_col].min())
    base['PERIODO_INICIO'] = ((base[ano_col] - ano_min) // periodo) * periodo + ano_min
    base['PERIODO_LABEL'] = (
        base['PERIODO_INICIO'].astype(str)
        + '-'
        + (base['PERIODO_INICIO'] + (periodo - 1)).astype(str)
    )

    # agrega por sexo e período
    agrupado = (
        base.groupby([raca_col, 'PERIODO_INICIO', 'PERIODO_LABEL'])
            .size()
            .reset_index(name='QUANTIDADE')
            .sort_values([raca_col, 'PERIODO_INICIO'])
    )

    # variação percentual entre períodos (vs período anterior)
    agrupado['VARIAÇÃO DO PERÍODO (%)'] = (
        agrupado.groupby(raca_col)['QUANTIDADE'].pct_change().mul(100)
    )

    # crescimento vs primeiro período
    def crescimento_base(s):
        primeiro = s.iloc[0]
        if primeiro == 0:
            return pd.Series([pd.NA] * len(s), index=s.index)
        return (s / primeiro - 1) * 100

    agrupado['CRESCIMENTO BASE'] = (
        agrupado.groupby(raca_col)['QUANTIDADE'].transform(crescimento_base)
    )

    # crescimento cumulativo (produto dos fatores)
    def crescimento_cumulativo(s):
        r = s.pct_change()
        fator = (1 + r).fillna(1.0).cumprod()
        return (fator - 1) * 100

    agrupado['CRESCIMENTO CUMULATIVO'] = (
        agrupado.groupby(raca_col)['QUANTIDADE'].transform(crescimento_cumulativo)
    )

    # escolha da métrica
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    elif modo == 'VARIAÇÃO ANUAL (%)' or modo == 'VARIAÇÃO DO PERÍODO (%)':
        metrica = 'VARIAÇÃO DO PERÍODO (%)'
        y_label = f'Variação vs período anterior (%)'
    elif modo == 'CRESCIMENTO BASE':
        metrica = 'CRESCIMENTO BASE'
        y_label = 'Crescimento em relação ao primeiro período (%)'
    else:
        metrica = 'CRESCIMENTO CUMULATIVO'
        y_label = 'Crescimento cumulativo (%)'

    # título
    tipo_titulo = (
        'todas as bolsas'
        if tipo_bolsa == 'TODOS OS TIPOS'
        else tipo_bolsa.lower()
            .replace('bolsa', 'bolsas')
            .replace('parcial', 'parciais')
            .replace('complementar', 'complementares')
            .replace('integral', 'integrais')
    )

    periodo_nome = 'ano' if periodo == 1 else f'períodos de {periodo} anos'
    if faixa_idade == 'TODAS AS IDADES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} por {periodo_nome} comparando raça'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} por {periodo_nome} na faixa etária {faixa_idade} anos'

    fig = px.line(
        agrupado,
        x='PERIODO_LABEL',
        y=metrica,
        color=raca_col,
        markers=True,
        hover_data={
            'PERIODO_LABEL': True,
            raca_col: True,
            'QUANTIDADE': ':.0f',
            'VARIAÇÃO DO PERÍODO (%)': ':.1f' if metrica == 'VARIAÇÃO DO PERÍODO (%)' else False,
            'CRESCIMENTO BASE': ':.1f' if metrica == 'CRESCIMENTO BASE' else False,
            'CRESCIMENTO CUMULATIVO': ':.1f' if metrica == 'CRESCIMENTO CUMULATIVO' else False,
        },
        color_discrete_map={
            'BRANCA': 'cyan',
            'PRETA': 'black',
            'PARDA': 'blue',
            'AMARELA': 'yellow',
            'INDÍGENA': 'red',
            'NÃO INFORMADA': 'magenta',
        },
        title=titulo,
    )

    # título
    tipo_titulo = 'todas as bolsas' if tipo_bolsa == 'TODOS OS TIPOS' else tipo_bolsa.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('complementar', 'complementares').replace('integral', 'integrais')
    if faixa_idade == 'TODAS AS IDADES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} comparando raças'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} na faixa etária {faixa_idade} anos'

    fig = px.line(
        agrupado,
        x='PERIODO_LABEL',
        y=metrica,
        color=raca_col,
        markers=True,
        hover_data={
            'PERIODO_LABEL': True,
            raca_col: True,
            'QUANTIDADE': ':.0f',
            'VARIAÇÃO DO PERÍODO (%)': ':.1f' if metrica == 'VARIAÇÃO DO PERÍODO (%)' else False,
            'CRESCIMENTO BASE': ':.1f' if metrica == 'CRESCIMENTO BASE' else False,
            'CRESCIMENTO CUMULATIVO': ':.1f' if metrica == 'CRESCIMENTO CUMULATIVO' else False,
        },
        color_discrete_map={
            'BRANCA': 'cyan',
            'PRETA': 'black',
            'PARDA': 'blue',
            'AMARELA': 'yellow',
            'INDÍGENA': 'red',
            'NÃO INFORMADA': 'magenta'
        },
        title=titulo,
    )

    # ordem da legenda
    ordem = ['AMARELA', 'BRANCA','INDÍGENA','PARDA', 'PRETA', 'NÃO INFORMADA']
    fig.data = tuple(sorted(fig.data, key=lambda tr: ordem.index(tr.name) if tr.name in ordem else len(ordem)))

    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title='Período',
        yaxis_title=y_label,
        legend_title='Raça',
        xaxis=dict(dtick=1),
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
        width=1200,
    )

    return fig
    
fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_idade_tipo_e_raca,
    controls={
        'faixa_idade': dropdown_faixa,
        'tipo_bolsa': dropdown_tipo,
        'modo': dropdown_modo,
        'periodo': dropdown_periodo,
    },
    folder=f'../figures/{crescimento_idade_tipo_e_raca.__name__}',
    default_title='crescimento_idade_tipo_e_raca',
)

Output()

## Crescimento de cada tipo de bolsa ao longo dos anos no país

In [21]:
def crescimento_bolsas_anos(tipo, regiao, modo):
    # Paleta opcional por tipo
    cores_por_tipo = {
        'TODOS OS TIPOS': 'yellow',
        'BOLSA INTEGRAL': 'yellow',
        'BOLSA PARCIAL 50%': 'yellow',
        'BOLSA COMPLEMENTAR 25%': 'yellow'
    }

    cores_por_regiao = {
        'PAÍS INTEIRO': 'yellow',
        'SUL': 'blue',
        'NORDESTE': 'magenta',
        'NORTE': 'green',
        'CENTRO-OESTE': 'black',
        'SUDESTE': 'red',
    }
        
    base = df_chars.copy()

    # Filtra tipo só se não for 'TODOS OS TIPOS'
    if tipo != 'TODOS OS TIPOS':
        base = base[base['TIPO DE BOLSA'] == tipo]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

    # Define métrica e label
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    else:
        metrica = 'CRESCIMENTO PERCENTUAL'
        y_label = 'Crescimento em relação ao primeiro ano (%)'

    # Função de crescimento percentual
    def crescimento_percentual(s):
        primeiro = s.iloc[0]
        if primeiro == 0:
            return pd.Series([pd.NA] * len(s), index=s.index)
        return (s / primeiro - 1) * 100

    # Cenário 1: país inteiro
    if regiao == 'PAÍS INTEIRO':
        agrupado = (
            base.groupby('ANO DE CONCESSÃO')
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values('ANO DE CONCESSÃO')
        )
        agrupado['TIPO DE BOLSA'] = tipo

        agrupado['CRESCIMENTO PERCENTUAL'] = crescimento_percentual(agrupado['QUANTIDADE'])

        cor_tipo = cores_por_tipo.get(tipo, '#1f77b4')

        fig = px.line(
            agrupado,
            x='ANO DE CONCESSÃO',
            y=metrica,
            markers=True,
            color_discrete_sequence=[cor_tipo],
            hover_data={
                'ANO DE CONCESSÃO': True,
                'TIPO DE BOLSA': True,
                'QUANTIDADE': ':.0f',
                'CRESCIMENTO PERCENTUAL': ':.1f' if modo != 'QUANTIDADE' else False,
            },
        )

    # Cenário 2: uma região específica
    elif regiao in df_chars['REGIÃO'].unique():
        base_reg = base[base['REGIÃO'] == regiao]
        if base_reg.empty:
            print('Sem dados para essa região.')
            return

        dados = (
            base_reg.groupby('ANO DE CONCESSÃO')
                    .size()
                    .reset_index(name='QUANTIDADE')
                    .sort_values('ANO DE CONCESSÃO')
        )
        agrupado['TIPO DE BOLSA'] = tipo
        agrupado['REGIÃO'] = regiao

        agrupado['CRESCIMENTO PERCENTUAL'] = crescimento_percentual(agrupado['QUANTIDADE'])

        cor_tipo = cores_por_tipo.get(tipo, '#1f77b4')

        fig = px.line(
            agrupado,
            x='ANO DE CONCESSÃO',
            y=metrica,
            markers=True,
            color_discrete_sequence=[cor_tipo],
            hover_data={
                'ANO DE CONCESSÃO': True,
                'TIPO DE BOLSA': True,
                'REGIÃO': True,
                'QUANTIDADE': ':.0f',
                'CRESCIMENTO PERCENTUAL': ':.1f' if modo != 'QUANTIDADE' else False,
            },
        )

    # Cenário 3: comparar regiões
    elif regiao == 'COMPARAR REGIÕES':
        dados = (
            base.groupby(['ANO DE CONCESSÃO', 'REGIÃO'])
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values(['REGIÃO', 'ANO DE CONCESSÃO'])
        )

        dados['CRESCIMENTO PERCENTUAL'] = (
            dados.groupby('REGIÃO')['QUANTIDADE'].transform(crescimento_percentual)
        )

        fig = px.line(
            dados,
            x='ANO DE CONCESSÃO',
            y=metrica,
            color='REGIÃO',
            markers=True,
            color_discrete_map=cores_por_regiao,  # aplica tua paleta
            hover_data={
                'ANO DE CONCESSÃO': True,
                'REGIÃO': True,
                'QUANTIDADE': ':.0f',
                'CRESCIMENTO PERCENTUAL': ':.2f' if modo != 'QUANTIDADE' else False,
            },
        )

    else:
        print('Opção de região inválida.')
        return

    tipo_titulo = 'todas as bolsas' if tipo == 'TODOS OS TIPOS' else tipo.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('complementar', 'complementares').replace('integral', 'integrais')
    if regiao == 'COMPARAR REGIÕES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} comparando regiões'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} no {regiao.lower()}'

    fig.update_layout(
        xaxis_title='ANO DE CONCESSÃO',
        yaxis_title=y_label,
        xaxis=dict(dtick=1),
        height=650,
        title=titulo,
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_bolsas_anos,
    controls={'tipo': dropdown_tipo, 'regiao': dropdown_regiao, 'modo': dropdown_modo},
    folder=f'../figures/{crescimento_bolsas_anos.__name__}',
    default_title='crescimento_bolsas_anos'
)

Output()

In [22]:
def crescimento_bolsas_anos(
    tipo_bolsa,
    regiao,                    # <- valor do dropdown
    modo,
    periodo=1,                 # 1=ano, 3=triênio, 5=quinquênio...
    ano_col='ANO DE CONCESSÃO',
    tipo_col='TIPO DE BOLSA',
    regiao_col='REGIÃO',
):
    cores_por_tipo = {
        'TODOS OS TIPOS': 'yellow',
        'BOLSA INTEGRAL': 'yellow',
        'BOLSA PARCIAL 50%': 'yellow',
    }

    cores_por_regiao = {
        'PAÍS INTEIRO': 'yellow',
        'SUL': 'blue',
        'NORDESTE': 'magenta',
        'NORTE': 'green',
        'CENTRO-OESTE': 'black',
        'SUDESTE': 'red',
    }

    base = df_chars.copy()
    base[ano_col] = base[ano_col].astype(int)

    # filtra tipo (se não for 'TODOS OS TIPOS')
    if tipo_bolsa != 'TODOS OS TIPOS':
        base = base[base[tipo_col] == tipo_bolsa]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

    # cria período (ano/triênio/quinquênio...)
    ano_min = int(base[ano_col].min())
    base['PERIODO_INICIO'] = ((base[ano_col] - ano_min) // periodo) * periodo + ano_min
    base['PERIODO_LABEL'] = (
        base['PERIODO_INICIO'].astype(str)
        + '-'
        + (base['PERIODO_INICIO'] + (periodo - 1)).astype(str)
    )
    # se periodo=1, fica mais bonito mostrar só o ano
    if periodo == 1:
        base['PERIODO_LABEL'] = base[ano_col].astype(str)

    # helper: calcula métricas por grupo
    def adicionar_metricas(df, col_grupo):
        df = df.sort_values([col_grupo, 'PERIODO_INICIO'])
        df['VARIAÇÃO DO PERÍODO (%)'] = df.groupby(col_grupo)['QUANTIDADE'].pct_change().mul(100)

        def crescimento_base(s):
            primeiro = s.iloc[0]
            if primeiro == 0:
                return pd.Series([pd.NA] * len(s), index=s.index)
            return (s / primeiro - 1) * 100

        def crescimento_cumulativo(s):
            r = s.pct_change()
            fator = (1 + r).fillna(1.0).cumprod()
            return (fator - 1) * 100

        df['CRESCIMENTO BASE'] = df.groupby(col_grupo)['QUANTIDADE'].transform(crescimento_base)
        df['CRESCIMENTO CUMULATIVO'] = df.groupby(col_grupo)['QUANTIDADE'].transform(crescimento_cumulativo)
        return df

    # escolhe métrica
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    elif modo == 'VARIAÇÃO DO PERÍODO (%)':
        metrica = 'VARIAÇÃO DO PERÍODO (%)'
        y_label = 'Variação vs período anterior (%)'
    elif modo == 'CRESCIMENTO BASE':
        metrica = 'CRESCIMENTO BASE'
        y_label = 'Crescimento em relação ao primeiro período (%)'
    else:
        metrica = 'CRESCIMENTO CUMULATIVO'
        y_label = 'Crescimento cumulativo (%)'

    # título base
    tipo_titulo = (
        'todas as bolsas'
        if tipo_bolsa == 'TODOS OS TIPOS'
        else tipo_bolsa.lower()
            .replace('bolsa', 'bolsas')
            .replace('parcial', 'parciais')
            .replace('complementar', 'complementares')
            .replace('integral', 'integrais')
    )
    periodo_nome = 'ano' if periodo == 1 else f'períodos de {periodo} anos'

    # CENÁRIO 1: país inteiro (agregado)
    if regiao == 'PAÍS INTEIRO':
        dados = (
            base.groupby(['PERIODO_INICIO', 'PERIODO_LABEL'])
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values('PERIODO_INICIO')
        )
        # cria coluna constante para calcular métricas
        dados['SÉRIE'] = tipo_bolsa
        dados = adicionar_metricas(dados, 'SÉRIE')

        cor = cores_por_tipo.get(tipo_bolsa, '#1f77b4')

        fig = px.line(
            dados,
            x='PERIODO_LABEL',
            y=metrica,
            markers=True,
            color_discrete_sequence=[cor],
            hover_data={
                'PERIODO_LABEL': True,
                'QUANTIDADE': ':.0f',
                'VARIAÇÃO DO PERÍODO (%)': ':.1f' if metrica == 'VARIAÇÃO DO PERÍODO (%)' else False,
                'CRESCIMENTO BASE': ':.1f' if metrica == 'CRESCIMENTO BASE' else False,
                'CRESCIMENTO CUMULATIVO': ':.1f' if metrica == 'CRESCIMENTO CUMULATIVO' else False,
            },
            title=f'{modo.lower().capitalize()} de {tipo_titulo} no país por {periodo_nome}',
        )

    # CENÁRIO 2: comparar regiões (várias linhas)
    elif regiao == 'COMPARAR REGIÕES':
        dados = (
            base.groupby([regiao_col, 'PERIODO_INICIO', 'PERIODO_LABEL'])
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values([regiao_col, 'PERIODO_INICIO'])
        )
        dados = adicionar_metricas(dados, regiao_col)

        fig = px.line(
            dados,
            x='PERIODO_LABEL',
            y=metrica,
            color=regiao_col,
            markers=True,
            color_discrete_map=cores_por_regiao,
            hover_data={
                'PERIODO_LABEL': True,
                regiao_col: True,
                'QUANTIDADE': ':.0f',
                'VARIAÇÃO DO PERÍODO (%)': ':.1f' if metrica == 'VARIAÇÃO DO PERÍODO (%)' else False,
                'CRESCIMENTO BASE': ':.1f' if metrica == 'CRESCIMENTO BASE' else False,
                'CRESCIMENTO CUMULATIVO': ':.1f' if metrica == 'CRESCIMENTO CUMULATIVO' else False,
            },
            title=f'{modo.lower().capitalize()} de {tipo_titulo} comparando regiões por {periodo_nome}',
        )

    # CENÁRIO 3: uma região específica
    else:
        base_reg = base[base[regiao_col] == regiao]
        if base_reg.empty:
            print('Sem dados para essa região.')
            return

        dados = (
            base_reg.groupby(['PERIODO_INICIO', 'PERIODO_LABEL'])
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values('PERIODO_INICIO')
        )
        dados['SÉRIE'] = regiao
        dados = adicionar_metricas(dados, 'SÉRIE')

        cor = cores_por_regiao.get(regiao, '#1f77b4')

        fig = px.line(
            dados,
            x='PERIODO_LABEL',
            y=metrica,
            markers=True,
            color_discrete_sequence=[cor],
            hover_data={
                'PERIODO_LABEL': True,
                'QUANTIDADE': ':.0f',
                'VARIAÇÃO DO PERÍODO (%)': ':.1f' if metrica == 'VARIAÇÃO DO PERÍODO (%)' else False,
                'CRESCIMENTO BASE': ':.1f' if metrica == 'CRESCIMENTO BASE' else False,
                'CRESCIMENTO CUMULATIVO': ':.1f' if metrica == 'CRESCIMENTO CUMULATIVO' else False,
            },
            title=f'{modo.lower().capitalize()} de {tipo_titulo} na região {regiao} por {periodo_nome}',
        )

    fig.update_layout(
        xaxis_title='Período',
        yaxis_title=y_label,
        xaxis_tickangle=-45,
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
        height=650,
    )
    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_bolsas_anos,
    controls={
        'tipo_bolsa': dropdown_tipo,
        'regiao': dropdown_regiao,
        'modo': dropdown_modo,
        'periodo': dropdown_periodo,
    },
    folder=f'../figures/{crescimento_bolsas_anos.__name__}',
    default_title='crescimento_bolsas_anos'
)

Output()

## 

## Mapa

In [23]:
# Carrega o geojson local (sem internet)
with open('brazil-states.geojson', encoding='utf-8') as f:
   geojson = json.load(f)

def mapa_bolsas_por_uf(ano, tipo):
   # Filtra por ano
   dados = df_chars[df_chars['ANO DE CONCESSÃO'] == ano].copy()
   
   if ano == 'TODOS OS ANOS':
      dados = df_chars.copy()

   # Filtra por tipo, se não for 'Todos'
   if tipo != 'TODOS OS TIPOS':
      dados = dados[dados['TIPO DE BOLSA'] == tipo]
   
   if dados.empty:
      print('Sem dados para essa combinação de ano e tipo.')
      return
   
   # Agrupa por UF e REGIAO
   mapa_df = (
      dados.groupby(['UF', 'REGIÃO'])
            .size()
            .reset_index(name='TOTAL')
            .sort_values('TOTAL', ascending=False)
   )

   # Título dinâmico
   titulo = f'Bolsas {tipo.lower().replace('bolsa', 'bolsas').replace('integral', 'integrais').replace('todos os tipos','')} do Prouni em {str(ano).lower().replace('todos os anos', 'de 2005 a 2019')}' if tipo != 'TODOS OS ANOS' else f'Bolsas por estado {tipo.lower().replace('bolsa', 'bolsas').replace('integral', 'integrais').replace('todos os tipos','')} do Prouni de 2005 a 2019'
   # Mapa coroplético por estado
   fig = px.choropleth(
      mapa_df,
      geojson=geojson,
      locations='UF',                  
      featureidkey='properties.sigla', 
      color='TOTAL',
      color_continuous_scale='Turbo',
      hover_data={'UF': True, 'REGIÃO': True, 'TOTAL': True},
      title=titulo,
   )

   fig.update_geos(
      fitbounds='locations',
      visible=False
   )

   fig.update_layout(
      height=500,
      coloraxis_colorbar=dict(title='TOTAL DE BOLSAS'),
      coloraxis_showscale=True,
      margin={'r':0, 't':50, 'l':0, 'b':0},
      width=1000,     
   ),

   return fig
   
fazer_plotly_widget_com_salvar(
    plot_fn=mapa_bolsas_por_uf,
    controls={'ano': dropdown_ano, 'tipo': dropdown_tipo},
    folder=f'../figures/{mapa_bolsas_por_uf.__name__}',
    default_title='mapa_bolsas_por_uf'
)

Output()